<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/wikipathway-data-visualization-basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WikiPathways Data Visualization Basic
## Yihang Xin, Kristina Hanspers and Alex Pico
## 2023-06-04

WikiPathways is a database of biological pathways maintained by and for the scientific community. Each pathway model in the database is annotated with database identfiers, which can be used for computation and data visualization.

* Basic workflow describing pathway import and data visualization and creating a legend.

# Installation
The following chunk of code installs the `py4cytoscape` module.

In [1]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)

In [2]:
#_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.7.0' # optional
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


Loading Javascript client ... 99071f83-23b6-45b4-81dc-52a3a7ac5df8 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

# Prerequisites
## In addition to this package (py4cytoscape latest version 1.7.0), you will need:

* Latest version of Cytoscape, which can be downloaded from https://cytoscape.org/download.html. Simply follow the installation instructions on screen.
* Complete installation wizard
* Launch Cytoscape

For this notebook, you’ll also need the WikiPathways app to access the WikiPathways database from within Cytoscape. 

Install the WikiPathways app from http://apps.cytoscape.org/apps/wikipathways

You can also install app inside Python notebook by running "py4cytoscape.install_app('Your App')"

# Import the required package

In [3]:
import py4cytoscape as p4c
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

## Import Cell Cycle pathway into Cytoscape.

In [14]:
cmd_list = ['wikipathways','import-as-pathway','id=WP179']
cmd = " ".join(cmd_list)
p4c.commands.commands_get(cmd) 

[]

The data we will be using for this workflow (E-GEOD-68086.csv) is from a study investigating the RNA-seq signatures of platelets from patients with various types of cancer, in an effort to develop a platelet-based diagnostic (Best et el., 2015). The data was downloaded from Expression Atlas.

In [5]:
df = pd.read_csv("https://cytoscape.github.io/cytoscape-tutorials/protocols/data/E-GEOD-68086.csv")
df.head()

,Gene ID,Gene Name,'breast carcinoma' vs 'normal' .foldChange,'breast carcinoma' vs 'normal'.pValue,'colorectal carcinoma' vs 'normal' .foldChange,'colorectal carcinoma' vs 'normal'.pValue,'glioblastoma' vs 'normal' .foldChange,'glioblastoma' vs 'normal'.pValue,'hepatobiliary carcinoma' vs 'normal' .foldChange,'hepatobiliary carcinoma' vs 'normal'.pValue,'non-small cell lung carcinoma' vs 'normal' .foldChange,'non-small cell lung carcinoma' vs 'normal'.pValue,'pancreatic adenocarcinoma' vs 'normal' .foldChange,'pancreatic adenocarcinoma' vs 'normal'.pValue
0,ENSG00000000003,TSPAN6,-1.2,8.524371e-03,-0.7,1.886303e-01,-1.6,0.000273,-0.9,0.144043,-2.1,3.970000e-07,-2.4,1.140000e-05
1,ENSG00000000005,TNMD,-2.4,1.890000e-04,-3.1,9.370000e-07,-2.5,0.000044,-1.0,0.193911,-2.1,8.710000e-05,-3.2,3.300000e-06
2,ENSG00000000419,DPM1,-1.0,3.210000e-05,-1.1,4.870000e-05,-0.9,0.002806,-0.3,0.410319,-0.8,4.849082e-03,-1.0,3.360000e-04
3,ENSG00000000457,SCYL3,-2.0,2.440000e-10,-2.0,4.980000e-10,-1.3,0.000105,-1.6,0.000035,-2.2,1.810000e-11,-3.0,3.780000e-18
4,ENSG00000000460,C1orf112,0.1,5.262086e-01,-0.1,7.400732e-01,-0.5,0.016092,0.2,0.366367,-0.9,1.150000e-05,-0.3,2.105133e-01


In [6]:
p4c.load_table_data(df, data_key_column="Gene ID", table_key_column="Ensembl")

/opt/anaconda3/lib/python3.8/site-packages/py4cytoscape/tables.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_subset[col] = col_val


'Success: Data loaded in defaultnode table'

## Data Visualization: Node Fill Color

We now have the data in Cytoscape, linked to the pathway, and ready to map to visual properties of the nodes in the network. For this workflow, we will choose data from one of the cancer types, "non-small cell lung carcinoma", and visualize the fold change as Node Fill Color.

In [7]:
p4c.set_node_color_mapping(**p4c.gen_node_color_map("'non-small cell lung carcinoma' vs 'normal' .foldChange", p4c.palette_color_brewer_d_RdBu(), style_name="WikiPathways"))

''

## Data Visualization: Node Border Width


We also have a significance measure for the fold change, which we can add to our visualization. For this, we will use both Node Border Width and Node Border Size to visualize the p value.



In [8]:
p4c.set_node_border_width_mapping("'non-small cell lung carcinoma' vs 'normal'.pValue",table_column_values=['0', '0.05'], widths=[5, 5], mapping_type='c', style_name='WikiPathways')

''

In [9]:
p4c.set_node_border_color_default("#5A5A5A", style_name='WikiPathways')

''

## Data Visualization: Node Border Color

In [10]:
p4c.set_node_border_color_mapping("'non-small cell lung carcinoma' vs 'normal'.pValue", ['0', '0.05', '0.05'], ['#FC0FC0', '#FC0FC0', '#5A5A5A'], 'c', style_name='WikiPathways')

''

We now have our example data visualized on the pathway, with fold change represented as a gradient node fill color and significance as node border color/width.



## Saving and Exporting Data Visualization

In [11]:
# Save as session
# p4c.session.save_session('wikipathway-data-visualization-basic')
# Export as PDF
# p4c.export_image('wikipathway-data-visualization-basic', type='PDF')

# Note: PNG, SVG, JPEG and PS are also supported
# p4c.export_image('differentially-expressed-genes', type='PNG')
# p4c.export_image('differentially-expressed-genes', type='JPEG')
# p4c.export_image('differentially-expressed-genes', type='SVG')
# p4c.export_image('differentially-expressed-genes', type='PS')